In [1]:
def load_annotations(anno_path):
        with open(anno_path, 'r') as f:
            lines = f.readlines()
        return lines

In [5]:
lines = load_annotations('data/exdark_train.txt')

In [20]:
bbox_list = lines[1].split(' ')[1:]
bbox_list

['174,64,329,284,7', '411,55,557,304,7', '4,144,141,433,7\n']

In [24]:
for bbox in bbox_list:
    print(list(map(int, bbox.split(',')))[-1])

7
7
7
